In [1]:
import networkx as nx
import copy
import gurobipy as gp
from gurobipy import GRB
from helpers import *
import time

# Instantiate processor

proc = Processor(4, 2, "HotspotConfiguration", 318.15)

# proc_cores is the number of processors per layer.
proc_layers = proc.nr_layers
proc_cores = proc.proc_per_layer

proc_total = proc.nr_pes
proc_config_path = proc.path_to_config


timestep = 1e-1

T_amb = proc.T_amb

coefficients = proc.coefficients


In [2]:
def create_model(graph : nx.DiGraph, T_crit : float, MIPGap = False):
    """
    Create and solve a mixed-integer programming model for TATS-ND-FP problem.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.
    - T_crit : float
      The critical temperature threshold for processors.
    - MIPGap : bool, optional
      If True, return the MIP gap along with other outputs. Default is False.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    - MIP gap : bool,optional
      The MIP gap, only returned if MIPGap is True.
    """
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    
    # Create a new model
    m = gp.Model("Matching")

    # Create variables
    makespan = m.addVar(name="makespan")
    start_time = [m.addVar(name=f"s({i})") for i in range(nr_tasks)]
    x = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name = f"x[{i},{p}]") for p in range(nr_processors)] for i in range(nr_tasks)] # x[i,p] == 1 iff task i scheduled on PE p
    b = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"b[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # b[i,j] == 1 iff task i finishes before task j starts
    o = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"o[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # o[i,j] == 1 iff task i overlaps with task j
    sig = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"sig[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # sig[i,j] == 1 iff task i overlaps with task j and st[i] <= st[j]
    sigp = [[[m.addVar(vtype=GRB.BINARY, name=f"sigp[{i},{j},{p}]") for p in range(nr_processors)] for j in range(nr_tasks)] for i in range(nr_tasks)] # sigp[i,j] == 1 iff sig[i,j] == 1 and x[j,p2] == 1 and x[i,p1] == 1
    d = [[m.addVar(lb=0, ub=1, vtype=GRB.BINARY, name=f"d[{i},{j}]") for j in range(nr_tasks)] for i in range(nr_tasks)] # d[i,j] == 1 iff task i and task j start at same time
    diff = [[m.addVar(lb=-float("inf"), name=f"diff[{i},{j}]") for j in range(i, nr_tasks)] for i in range(nr_tasks)] # diff[i,j] == start_time[i] - start_time[j]
    absdiff = [[m.addVar(name=f"absdiff[{i},{j}]") for j in range(i, nr_tasks)] for i in range(nr_tasks)] # absdiff[i,j] == |start_time[i] - start_time[j]|

    # Set objective: minimize makespan
    m.setObjective(1.0 * makespan, GRB.MINIMIZE)

    # Define makespan
    m.addConstrs((start_time[i] + task_lengths[i] <= makespan for i in range(nr_tasks)), name="ms_def")

    # Task is planned on exactly one processor:
    m.addConstrs((sum(x[i][p] for p in range(nr_processors)) == 1 ) for i in range(nr_tasks))

    # Define b[i,j] and o[i,j]
    M = sum(task_lengths)  # Large constant: Worst case execution time, i.e. executing all tasks on the slowest core.
    m.addConstrs((start_time[i] + task_lengths[i] <= start_time[j] + M * (1 - b[i][j]) + M * o[i][j]) for i in range(nr_tasks) for j in range(nr_tasks))
    m.addConstrs((start_time[i] + M * (b[i][j] + o[i][j]) >= start_time[j] + task_lengths[j]) for i in range(nr_tasks) for j in range(nr_tasks))

    # Define sig[i][j]
    m.addConstrs((M * (1 - o[i][j]) + M * (1 - sig[i][j]) + M * d[i][j] + start_time[i] >= start_time[j] + 0.5) for i in range(nr_tasks) for j in range(nr_tasks))
    m.addConstrs(((1 - o[i][j]) + sig[i][j] + sig[j][i] >= 1) for i in range(nr_tasks) for j in range(nr_tasks))
    # add constr if st[i] == st[j] => sig[i][j] == sig[j][i] = 1 wrs

    # Define sigp[i][j][p]
    m.addConstrs(((sig[i][j] + x[j][p]) + 3 * (1 - sigp[i][j][p]) <= 4) for i in range(nr_tasks) for j in range(nr_tasks) for p in range(nr_processors) )

    # Define delta[u][v]
    m.addConstrs((diff[i][j] == start_time[i] - start_time[i + j]) for i in range(nr_tasks) for j in range(nr_tasks - i))
    m.addConstrs((absdiff[i][j] == gp.abs_(diff[i][j])) for i in range(nr_tasks) for j in range(nr_tasks - i))
    m.addConstrs((absdiff[i][j] <= 1/2 +  M * (1 - d[i][i + j])) for i in range(nr_tasks) for j in range(nr_tasks - i))
    m.addConstrs((absdiff[i][j] >= 1 -  M * d[i][i + j]) for i in range(nr_tasks) for j in range(nr_tasks - i))
    m.addConstrs(d[i][j] == d[j][i] for i in range(nr_tasks) for j in range(nr_tasks - i))
    m.addConstrs((2 * (1 - d[i][j]) + sig[i][j] + sig[j][i] >= 2) for i in range(nr_tasks) for j in range(nr_tasks))


    # If  two task on same processor, then one of them comes before the other
    m.addConstrs((x[i][p] + x[j][p] + o[i][j] <= 2) for i in range(nr_tasks) for j in range(nr_tasks) for p in range(nr_processors) if i != j)

    # Do not overheat
    m.addConstrs((sum(sum(coefficients[p1][p2] * sigp[u][v][p2] * task_power[v] for v in range(nr_tasks)) for p2 in range(nr_processors)) <= T_crit - T_amb) for u in range(nr_tasks) for p1 in range(nr_processors) )

    # Optimize model
    m.optimize()


    tasks_per_proc = [[] for _ in range(nr_processors)]

    # Print the values of all variables
    for i in range(nr_tasks):
        for p in range(nr_processors):
            if x[i][p].X == 1.0:
                print("Task", i, "is mapped on PE", p, ". start time:", start_time[i].X, ", finish time:", start_time[i].X + task_lengths[i])
                tasks_per_proc[p].append((i, start_time[i].X, start_time[i].X + task_lengths[i], task_power[i]))
    
    for u in range(nr_tasks):
        for p1 in range(nr_processors):
            print(f"T[{u},{p1}] = ", T_amb + sum(sum(coefficients[p1][p2] * sigp[u][v][p2].X * task_power[v] for v in range(nr_tasks)) for p2 in range(nr_processors)))
        
    ms = round(makespan.X)
    gap = float(m.MIPGap)

    m.dispose()
    
    if MIPGap:
        return ms, tasks_per_proc, gap
    return ms, tasks_per_proc



In [3]:
def splitting_heuristic(graph : nx.DiGraph, T_crit : float, group_size : int):
    """
    Splits a directed graph into smaller subgraphs, processes each subgraph using the MILP formulation of TATS-ND-FP, 
    and combines the results.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.
    - T_crit : float
      The critical temperature threshold for processors.
    - group_size (int): 
      The size of each group into which the tasks are divided.

    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    class Graph:
        def __init__(self) -> None:
            self.nodes = dict()
    nr_tasks = len(graph.nodes)
    nr_groups = nr_tasks // group_size
    if nr_tasks % group_size != 0:
        nr_groups += 1
    
    complete_tpp = [[] for _ in range(proc_total)]
    complete_ms = 0
    for i in range(nr_groups):
        modified_graph = Graph()
        index = 0
        for r in range(group_size * i, min(group_size * (i + 1), nr_tasks)):
            modified_graph.nodes[index] = graph.nodes[r]
            index += 1
        print(modified_graph.nodes)
        ms, tpp = create_model(modified_graph, T_crit)
        for p in range(proc_total):
            for t, st, ft, pow in tpp[p]:
                complete_tpp[p].append((t + i * group_size, st + complete_ms, ft + complete_ms, pow))
        complete_ms += ms
    return complete_ms , complete_tpp

def combine_tpp(tpp1, tpp2):
    """
    Combines two lists of task per processor (TPP) data structures 

    Parameters:
    -----------
    tpp1 : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    tpp2 : list[list[tuple[int, float, float, float]]] 
      The second tpp, structured similarly to `tpp1`.

    Returns:
    list[list[tuple[int, float, float, float]]] 
      tpp1 with tpp2 concatinated to it (with updated start and finish times)
    """
    ms1 = 0
    for ptpp in tpp1:
        for i, st, ft, pow in ptpp:
            ms1 = max(ms1, ft)
    ms2 = 0
    complete_tpp = copy.deepcopy(tpp1)
    for p in range(len(tpp2)):
        for i, st, ft, pow in tpp2[p]:
            ms2 = max(ms2, ft)
            complete_tpp[p].append((i, st + ms1, ft + ms1, pow))
 
    return ms1 + ms2, complete_tpp

In [4]:
def greedy_scheduling(graph : nx.DiGraph, T_crit: float):
    """
    Greedy algorithm for the TATS-ND-FP problem.

    Parameters:
    -----------
    - graph : nx.DiGraph
      A task graph. Each node should have the following attributes:
      - 'weight': The duration of the task.
      - 'power_cons': The power consumption of the task.
    - T_crit : float
      The critical temperature threshold for processors.
    
    Returns:
    --------
    - makespan : int
      The total time required to complete all tasks.
    - tasks_per_proc : list[list[tuple[int, float, float, float]]] 
      A list of lists, where each inner list contains tuples representing the tasks scheduled on each processor. Each tuple contains:
      - task number : int
      - start time : float
      - finish time : float
      - power consumption : float
    """
    tpp = [[] for _ in range(proc_total)]
    nr_tasks = len(graph.nodes)
    nr_processors = proc_total
    task_lengths = [graph.nodes[i]['weight'] for i in graph.nodes]
    task_power = [graph.nodes[i]['power_cons'] for i in graph.nodes]
    
    makespan = 0

    def power_consumption_of_core(time, core):
        for t, st, ft, pow in tpp[core]:
            if round(st) <= time < round(ft):
                return pow
        return 0


    for i in range(nr_tasks):
        opt_t = None
        opt_p = None
        for p in range(nr_processors):
            possible_time_stamps = []
            for t in range(makespan):
                if power_consumption_of_core(t, p) == 0:
                    power_trace = [power_consumption_of_core(t, x) for x in range(nr_processors)]
                    power_trace[p] = task_power[i]
                    for p2 in range(nr_processors):
                        if proc.temperature(p2, power_trace) > T_crit:
                            break
                    else:
                        possible_time_stamps.append(t)
            for t in possible_time_stamps:
                for t_run in range(t, t + task_lengths[i]):
                    if t_run not in possible_time_stamps and t_run <= makespan:
                        break
                else:
                    if opt_t == None or t < opt_t:
                        opt_t = t
                        opt_p = p
            else:
                if opt_t == None:
                    opt_t = makespan
                    opt_p = p
        
        tpp[opt_p].append((i, opt_t, opt_t + task_lengths[i], task_power[i]))
        makespan = max(makespan, opt_t + task_lengths[i])
    
    return makespan, tpp
    

In [ ]:

output_file_normal = "output_normal_model_approx.txt"
output_file_splitting = "output_splitting_model.txt"
output_file_greedy = "output_greedy_model.txt"

# Run Experiments 
prev_time = 0
prev_makespan = 0
prev_tpp = [[] for _ in range(proc_total)]
for i in range(1, 10, 1):

    # Run experiments for MILP
    task_graph = create_random_graph(i)
    start = time.time()
    ms, tpp, gap = create_model(task_graph, 400, True)
    proc.tpp_to_power_trace(tpp, ms, output_file_normal, 1e-1)
    t_max = proc.compute_max_ptrace_temp(output_file_normal)
    f = open(output_file_normal, 'a')
    f.write(f"Nr tasks {i}, Time {time.time() - start}, Makespan {ms}, Max Temp {t_max} , GAP {gap * 100}%\n")
    f.close()
    if gap * 100 > 25:
        break

    # Run experiments for Greedy Algorithm
    start = time.time()
    ms, tpp = greedy_scheduling(task_graph, 400)
    proc.tpp_to_power_trace(tpp, ms, output_file_greedy, 1e-1)
    end = time.time()
    t_max, top1p = proc.compute_max_ptrace_temp(output_file_greedy, True)
    f = open(output_file_greedy, 'a')
    f.write(f"Nr tasks {i}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Top 1 percent highs {top1p} \n")
    f.close()
    
    

    # Run experiments for Splitting Algorithm
    start = time.time()
    ms, tpp = splitting_heuristic(task_graph, 400, 10)
    proc.tpp_to_power_trace(tpp, ms, output_file_splitting, 1e-1)
    end = time.time()
    t_max, top1p = proc.compute_max_ptrace_temp(output_file_splitting, True)
    f = open(output_file_splitting, 'a')
    f.write(f"Nr tasks {i}, Time {end - start}, Makespan {ms}, Max Temp {t_max}, Top 1 percent highs {top1p} \n")
    f.close()
    
